# Day 20

## Part1

In [1]:
import numpy as np

### Read input

In [2]:
# Open file
#f = open("Day20.txt", "r")
f = open("input_data/Day20.txt", "r")

# Initialize our dictionary, which consists of a key of tile number and a value of
# a numpy array containing the image and a flag whether it is used or not
tiles = {}

line = f.readline()
while line:

    # Process the tile number
    line = line.strip('\n')
    line = line.strip('Tile')
    line = line.strip(':')
    line = line.strip(' ')
    tile_num = line

    # Now we read each line of the tile
    # initialize our numpy array
    tile_image = np.zeros([10, 10], dtype='int')

    line = f.readline()
    num_row=0
    while (line != '\n') & (line != ''):
        line = line.strip('\n')
        tile_image[num_row] = np.array([True if i=='#' else False for i in line])
        num_row += 1
        line = f.readline()

    # we're done reading this tile so store it in our dictionary
    tiles[tile_num] = {'image': tile_image, 'edges': {'0':'', '90':'', '180':'', '270':''}, 'matches': {}}

    # we've gotten a blank line so read the next tile number
    line = f.readline()

# Close file
f.close()

# This is the dimension of our tiles (10x10)
num_dim = 10

# This is the dimension of our cropped tiles (8x8)
crop_dim = num_dim -2

# This is the dimension of our puzzle 
puzzle_dim = int(np.sqrt(len(tiles.keys())))

# Directions
directions = ['0','90','180','270']

# Number of circuits (this is the number of times we need to build edges as we work our way inside)
num_circuits = round(puzzle_dim / 2)

# initialize our puzzle
puzzle = np.zeros([puzzle_dim*crop_dim, puzzle_dim*crop_dim], dtype='int')
puzzle_ids = []
for i in range(puzzle_dim):
    puzzle_ids.append(['0' for i in range(puzzle_dim)])

### Function Definitions

In [3]:


def match_tiles(tile1, tile2):
    '''takes two tiles and matches all free edges on tile1 with any free edges on tile2.
    Updates both tile1 and tile2 if a match is found'''

    # Check each available edge on tile1
    for dir1 in directions:
        if tiles[tile1]['edges'][dir1]=='':
            for dir2 in directions:
                if tiles[tile2]['edges'][dir2]=='':
                    if match_edges(tile1, dir1, tile2, dir2):
 #                       print('found a match with ', tile1, dir1, tile2, dir2)
                        tiles[tile1]['edges'][dir1] = tile2
                        tiles[tile2]['edges'][dir2] = tile1
                        return True
    return False


def get_edge(tile_num, dir):
    '''takes a tile and a direction and returns an edge'''

    if dir=='90':
        return tiles[tile_num]['image'][0]
    elif dir=='0':
        return tiles[tile_num]['image'][:,num_dim-1]
    elif dir == '270':
        return tiles[tile_num]['image'][num_dim-1]
    elif dir == '180':
        return tiles[tile_num]['image'][:,0]


def match_edges(tile1, dir1, tile2, dir2):
    '''takes two tiles and two directions and returns True if a match'''
 
    edge1 = get_edge(tile1, dir1)
    edge2 = get_edge(tile2, dir2)
    
    # Try edge flipped or not
    if np.sum(edge1 == edge2) == num_dim:
        return True
    elif np.sum(edge1 == np.flip(edge2)) == num_dim:
        return True
    else:
        return False
    
    

### Main Loop

In [4]:
# We'll loop through our tiles, matching with each tile except itself

for tile1 in tiles.keys():
    
    for tile2 in tiles.keys():
        
        if (tile1 != tile2):
            match_tiles(tile1, tile2)
    

In [5]:
# Now let's count corner, edge, and middle pieces

corners = []
edges = []
middles = []

for tile in tiles.keys():
    
    matched_edges = 0
    for dir in directions:
        
        if tiles[tile]['edges'][dir] != '':
            matched_edges += 1

    if matched_edges == 2:
        corners.append(tile)
    elif matched_edges == 3:
        edges.append(tile)
    elif matched_edges == 4:
        middles.append(tile)
        
print('number of corners: ', len(corners))
print('number of edges: ', len(edges))
print('number of middles: ', len(middles))

prod = 1
for tile in corners:
    prod *= int(tile)
    
print('product of corner tiles:', prod)

number of corners:  4
number of edges:  40
number of middles:  100
product of corner tiles: 63187742854073


## Part 2

### Function Definitions

In [6]:
def crop_image(image):
    '''takes a num_dim square image and removes the borders'''
    
    cropped_image = np.empty([num_dim-2, num_dim-2], dtype='bool')
    for i in range(num_dim-2):
        for j in range(num_dim-2):
            cropped_image[i][j] = image[i+1][j+1]
            
    return cropped_image

In [7]:
def rotate_tile(tile):
    '''rotates a tile CCW 90'''
    
    temp = tiles[tile]['edges']['270']
    tiles[tile]['edges']['270'] = tiles[tile]['edges']['180']
    tiles[tile]['edges']['180'] = tiles[tile]['edges']['90']    
    tiles[tile]['edges']['90'] = tiles[tile]['edges']['0']
    tiles[tile]['edges']['0'] = temp
    
    tiles[tile]['image'] = np.rot90(tiles[tile]['image'])

    return


In [8]:
def flip_tile(tile, axis):
    '''flips a tile about the vertical axis'''
    
    if axis == 'N-S':
        temp = tiles[tile]['edges']['180']
        tiles[tile]['edges']['180'] = tiles[tile]['edges']['0']    
        tiles[tile]['edges']['0'] = temp
    
        tiles[tile]['image'] = np.flip(tiles[tile]['image'], axis=1)
    
    else: # flip along E-W axis
        temp = tiles[tile]['edges']['90']
        tiles[tile]['edges']['90'] = tiles[tile]['edges']['270']    
        tiles[tile]['edges']['270'] = temp
    
        tiles[tile]['image'] = np.flip(tiles[tile]['image'], axis=0)
        
    return

In [9]:
def orient_next_tile(tile1, dir1):
    '''finds tile2 that matches tile1 along dir1 and re-orients it to line up with tile1
    returns tile2'''
    
    # find which tile matches this edge of tile1
    tile2 = tiles[tile1]['edges'][dir1]
    # find which edge of tile2 matches this edge of tile1
    for direction in directions:
        if tiles[tile2]['edges'][direction] == tile1:
            dir2 = direction
    
    diff = (int(dir2) - int(dir1))%360
    if diff == 0:
        # rotate 2x
        rotate_tile(tile2)
        rotate_tile(tile2)
        dir2 = str((180+int(dir2))%360)
    if diff == 90:
        # rotate 1x
        rotate_tile(tile2)
        dir2 = str((90+int(dir2))%360)
    if diff == 180:
        # don't rotate
        pass
    if diff == 270:
        # rotate 3x
        rotate_tile(tile2)
        rotate_tile(tile2)
        rotate_tile(tile2)
        dir2 = str((270+int(dir2))%360)
        
    # Now check if edges match propertly or if edge 2 needs to be reversed
    # if it does we flip the tile along the appropriate axis

    # get the edges
    edge1 = get_edge(tile1, dir1)
    edge2 = get_edge(tile2, dir2)
    
    if np.sum(edge1 == np.flip(edge2)) == 10:
        if dir1 in ['0','180']:
            flip_tile(tile2, 'E-W')
        else:
            flip_tile(tile2, 'N-S')

    return tile2

In [10]:
def get_corner_tile(circuit_num):
    '''starts with a circuit num and finds a corner piece'''
    
    if circuit_num == 0:
        # we have to initialize it
        
        # test case
        # we're going to start with a corner and rotate it so it's N and W edges are unconnected
        # and it's oriented the way we want
#        corner_tile='1951'
#        flip_tile(corner_tile, 'E-W')
        
        # Real data
        corner_tile = '3209'
        
    else:
        
        # we grab puzzle piece above this location to find out our tile number
        upper_tile = puzzle_ids[circuit_num-1][circuit_num]
        print(upper_tile)
        
        # Orient the tile along the bottom edge
        corner_tile = orient_next_tile(upper_tile, '270')
            
    # fill in puzzle id and puzzle with the corner image
    place_tile(corner_tile, circuit_num, circuit_num)   

    return corner_tile



In [11]:
def build_circuit(circuit_num, corner_tile):
    '''builds a circuit, starting in NW corner with a corner tile'''
    
    # if we have an odd puzzle dimension and we're at the last circle, we just have a middle
    # piece and we're done
    
    if (circuit_num == round(puzzle_dim/2)) and (puzzle_dim%2!=0):
        return
    

    i, j = circuit_num, circuit_num
    print('Circuit Number:', circuit_num)
    
    tile1 = corner_tile
    
    for j in range(circuit_num+1,puzzle_dim-circuit_num):

 #       print('Northern:', i,j)
        # build along northern edge
        tile2 = orient_next_tile(tile1, '0')
 #       print(tile1, tile2)
        
        # place tile
        place_tile(tile2, i, j)
        
        # Now move on
        tile1 = tile2
        
    for i in range(circuit_num+1,puzzle_dim-circuit_num):
        # build along eastern edge
 #       print('Eastern:', i,j)
        tile2 = orient_next_tile(tile1, '270')
 #       print(tile1, tile2)

        # place tile
        place_tile(tile2, i, j)
    
        # Now move on
        tile1 = tile2

    for j in range(puzzle_dim-circuit_num-2, circuit_num-1, -1):
 
        # build along southern edge
 #       print('Southern:', i,j)
        tile2 = orient_next_tile(tile1, '180')
 #       print(tile1, tile2)

        # place tile
        place_tile(tile2, i, j)
    
        # Now move on
        tile1 = tile2

    for i in range(puzzle_dim-circuit_num-2, circuit_num, -1):
        # build along southern edge
 #       print('Western:', i,j)
        tile2 = orient_next_tile(tile1, '90')
 #       print(tile1, tile2)

        # place tile
        place_tile(tile2, i, j)
   
        # Now move on
        tile1 = tile2

    return

In [12]:
def place_tile(tile, i, j):
    '''places a tile into position i, j in our puzzle and puzzle id'''
    
    puzzle_ids[i][j] = tile
    start_i = i * crop_dim
    start_j = j * crop_dim
    puzzle[start_i:start_i+crop_dim,start_j:start_j+crop_dim] = crop_image(tiles[tile]['image'])

    return


### Main Loop

In [13]:
# Define our sea monster

sea_monster = np.zeros([3,20], dtype='int')
sea_monster[0,18]=1
sea_monster[1,0]=1
sea_monster[1,5]=1
sea_monster[1,6]=1
sea_monster[1,11]=1
sea_monster[1,12]=1
sea_monster[1,17]=1
sea_monster[1,18]=1
sea_monster[1,19]=1
sea_monster[2,1]=1
sea_monster[2,4]=1
sea_monster[2,7]=1
sea_monster[2,10]=1
sea_monster[2,13]=1
sea_monster[2,16]=1

# Define number of matches we need
matches = 15

In [14]:
for i in range(round(puzzle_dim/2)):
    
    corner_tile = get_corner_tile(i)

    build_circuit(i, corner_tile)

print(puzzle_ids)

Circuit Number: 0
1193
Circuit Number: 1
2549
Circuit Number: 2
3673
Circuit Number: 3
3257
Circuit Number: 4
2131
Circuit Number: 5
[['3209', '1193', '2441', '2063', '3931', '3581', '3727', '1987', '1163', '3733', '2963', '3803'], ['2237', '1409', '2549', '1663', '2417', '3067', '2381', '2731', '2729', '1993', '2389', '2161'], ['3767', '1997', '3877', '3673', '2087', '2711', '3307', '1123', '3761', '3793', '3347', '1231'], ['2777', '3719', '1153', '1361', '3257', '2749', '2347', '1559', '3541', '2017', '2383', '2473'], ['2351', '3517', '1979', '1933', '3407', '2131', '2377', '1607', '1489', '2411', '1613', '1321'], ['1039', '2917', '2203', '2767', '3607', '1693', '2819', '3301', '3191', '1453', '2897', '1823'], ['2281', '3457', '3449', '2879', '3571', '1117', '3911', '3389', '1237', '2707', '2531', '1279'], ['2251', '3511', '3691', '3323', '3163', '1171', '3259', '2083', '1481', '1777', '1033', '3079'], ['2137', '1753', '3559', '1801', '1427', '2647', '1187', '2969', '2297', '1259', '

In [15]:


def search_image():
    '''takes the current puzzle image and searches it for sea monsters.  Returns the number
    of monsters found'''
    
    sea_monsters = 0
    for i in range(crop_dim*puzzle_dim-2): # loop down rows
        for j in range(crop_dim*puzzle_dim-19): # loop across row
            if (np.sum(np.sum(np.logical_and(sea_monster, puzzle[i:i+3,j:j+20]))) == 15):
                sea_monsters += 1
    return sea_monsters



In [16]:


def search_all_orientations():
    '''searches all orientations of the puzzle and returns the number of sea monsters
    if it finds sea monsters'''

    global puzzle
    # Search puzzle image
    sea_monsters = search_image()
    if sea_monsters>0:
        return sea_monsters
    
    # Rotate and search 
    puzzle = np.rot90(puzzle)
    sea_monsters = search_image()
    if sea_monsters>0:
        return sea_monsters
 
    # Rotate and search 
    puzzle = np.rot90(puzzle)
    sea_monsters = search_image()
    if sea_monsters>0:
        return sea_monsters
 
    # Rotate and search 
    puzzle = np.rot90(puzzle)
    sea_monsters = search_image()
    if sea_monsters>0:
        return sea_monsters
 
    # Flip and search
    puzzle = np.flip(puzzle, axis=1)
    sea_monsters = search_image()
    if sea_monsters>0:
        return sea_monsters
 
    # Rotate and search 
    np.rot90(puzzle)
    sea_monsters = search_image()
    if sea_monsters>0:
        return sea_monsters
 
    # Rotate and search 
    np.rot90(puzzle)
    sea_monsters = search_image()
    if sea_monsters>0:
        return sea_monsters
 
    # Rotate and search 
    np.rot90(puzzle)
    sea_monsters = search_image()
    if sea_monsters>0:
        return sea_monsters

    return -1



In [20]:
search_all_orientations()

18

In [21]:
search_image()

18

In [22]:
# We want to know how many 1's are in the area without monsters
np.sum(np.sum(puzzle))-18*15

2152